<a href="https://colab.research.google.com/github/TemrinaDarya/turbines-2022/blob/gas_turbines_hw1/gas_turbines_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from typing import Dict 
class GasField:
    cal_vals = {
        'ch4': 358.2,
        'c2h6': 637.46,
        'c3h8': 860.05,
        'c4h10': 1185.8,
        'c5h12': 1461.7}
   
    heat_co2 = {
        'three': 4.5784 * 10 ** -11,
        'two': 1.51719 * 10 ** -7,
        'one': 2.50113 * 10 ** -4,
        'zero': 0.382325}
    
    heat_n2 = {
        'three': 2.24553 * 10 ** -11,
        'two': 4.85082 * 10 ** -7,
        'one': 2.90598 * 10 ** -4,
        'zero': 0.309241}
    
    heat_h2o = {
        'three': 2.10956 * 10 ** -11,
        'two': 4.9732 * 10 ** -7,
        'one': 2.60629 * 10 ** -4,
        'zero': 0.356691}
    
    heat_air = {
        'three': 2.1717 * 10 ** -11,
        'two': 4.19344 * 10 ** -7,
        'one': 8.00891 * 10 ** -4,
        'zero': 0.315027}
    
    _stoichiometric_coefficient = 0.0476
    _excess_air_coefficient = 1.293
    _alpha = 3.131
    _temp = 1200 
    _coefficient_heat = 4.1868
    def __init__(self,
                 name: str,
                 ch4: float = 0.0,
                 c2h6: float = 0.0,
                 c3h8: float = 0.0,
                 c4h10: float = 0.0,
                 c5h12: float = 0.0,
                 n2: float = 0.0,
                 co2: float = 0.0,
                 o2: float = 0.0,
                 co: float = 0.0,
                 h2: float = 0.0,
                 ):
        if round(ch4 + c2h6 + c3h8 + c4h10 + c5h12 + n2 + co2 + o2 + co + h2, 6) != 100:
          raise ValueError(f'Критерий равенства 100% не выполнен', round(100 - (ch4 + c2h6 + c3h8 + c4h10 + c5h12 + n2 + co2 + o2 + co + h2), 6), '%')

        self.name = name
        self.__ch4 = ch4 / 100
        self.__c2h6 = c2h6 / 100
        self.__c3h8 = c3h8 / 100
        self.__c4h10 = c4h10 / 100
        self.__c5h12 = c5h12 / 100
        self.__n2 = n2 / 100
        self.__co2 = co2 / 100
        self.__o2 = o2 / 100
        self.__co = co / 100
        self.__h2 = h2 / 100

    @property
    def net_calorific_value(self):
        calorific_value = self.cal_vals['ch4'] * self.__ch4 + self.cal_vals['c2h6'] * self.__c2h6 + \
                          self.cal_vals['c3h8'] + self.__c3h8 + self.cal_vals['c4h10'] * self.__c4h10
        return calorific_value
    @property
    def V0(self):
        V0 = self._stoichiometric_coefficient * (
                2 * self.__ch4 + 3.5 * self.__c2h6 + 5 + self.__c3h8 + 6.5 * self.__c4h10 + 8 * self.__c5h12
        )
        return V0
    @property
    def V0n2(self):
        V0n2 = 0.79 * self.V0 + 0.01 * self.__n2
        return V0n2

    @property
    def V0ro2(self):
        V0ro2 = 0.01 * (self.__co2  + self.__ch4 + 2 * self.__c2h6 + 3 * self.__c3h8 + 4 * self.__c4h10 + 5 * self.__c5h12)
        return V0ro2
    @property
    def Vh20(self):
        V0h2o = 0.01 * (2 * self.__ch4 + 3 * self.__c2h6 + 4 * self.__c3h8 + 5 * self.__c4h10 + 6 * self.__c5h12)
        return V0h2o
    @property
    def Vh2o(self):
        Vh2o = self.Vh20 + 0.0161 * (self._alpha - 1) * self.V0
        return Vh2o
    @property
    def Vg(self):
        Vg = self.V0ro2 + self.V0n2 + self.Vh2o + (self._alpha - 1) * self.V0
        return Vg
    @property
    def cco2(self):
        cco2 = self._coefficient_heat * (self. heat_co2['three'] * self._temp ** 3 - self. heat_co2['two'] * self._temp ** 2 + 
                                       self. heat_co2['one'] * self._temp + self. heat_co2['zero'])
        return cco2
    @property
    def cn2(self):    
        cn2 = self._coefficient_heat * (self. heat_n2['three'] * self._temp ** 3 - self. heat_n2['two'] * self._temp ** 2 + 
                                       self. heat_n2['one'] * self._temp + self. heat_n2['zero'])
        return cn2
    @property
    def ch2o(self):
        ch2o = self._coefficient_heat * (self. heat_h2o['three'] * self._temp ** 3 - self. heat_h2o['two'] * self._temp ** 2 + 
                                       self. heat_h2o['one'] * self._temp + self. heat_h2o['zero'])
        return ch2o
    @property
    def cair(self):   
        cair = self._coefficient_heat * (self. heat_air['three'] * self._temp ** 3 - self. heat_air['two'] * self._temp ** 2 + 
                                       self. heat_air['one'] * self._temp + self. heat_air['zero'])
        return cair 
    @property
    def h0(self):
        h0 = (self.V0ro2 * self.cco2 + self.V0n2 * self.cn2 + self.Vh2o * self.ch2o) * self._temp
        return h0
    @property
    def hair0(self):
        hair0 = (self.V0 * self.cair) * self._temp
        return hair0
    @property
    def h(self):
        h = self.h0 + (self._alpha - 1) * self.hair0
        return h

    def __repr__(self):
        return f'<GasField {self.name} ch4:{self.__ch4} c2h6:{self.__c2h6}>'


gas_fields_list = [
    {'name': 'Уренгой', 'ch4': 98.4, 'c2h6': 0.1, 'n2': 1.2, 'co2': 0.3},
    {'name': 'Ямбургское', 'ch4': 98.6, 'c2h6': 0.1, 'n2': 1.2, 'co2': 0.1},
    {'name': 'Заполярное','ch4': 99.3, 'c2h6': 0.1, 'n2': 0.4,'co2': 0.2},
    {'name': 'Медвежье','ch4': 97.622, 'c2h6': 1.0, 'c3h8': 0.1, 'c4h10': 0.1, 'c5h12': 0.178, 'n2': 0.5,'co2': 0.5},
    {'name': 'Оренбургское', 'ch4': 84.37, 'c2h6': 4.6, 'c3h8': 1.64, 'c4h10': 0.81, 'c5h12': 1.88, 'n2': 4.34,'co2': 0.87, 'h2': 1.49},
    {'name': 'Вуктыльское', 'ch4': 75.1, 'c2h6': 8.9, 'c3h8': 3.6, 'c4h10': 1.5, 'c5h12': 6.4, 'n2': 4.4, 'co2': 0.1},
    {'name': 'Шебелинское', 'ch4': 92.07, 'c2h6': 3.26, 'c3h8': 0.59, 'c4h10': 0.18, 'c5h12': 0.6, 'n2': 1.3, 'co2': 2.00},
    {'name': 'Газлинское', 'ch4': 96.9, 'c2h6': 1.74, 'c3h8': 0.04, 'c4h10': 0.01, 'c5h12': 0.01, 'n2': 1.15, 'co2': 0.15},
    {'name': 'Астраханское', 'ch4': 90.48, 'c2h6': 2.07, 'c3h8': 0.99, 'c4h10': 1.75, 'c5h12': 0.61, 'n2': 3.45, 'co2': 0.65}
]

gas_fields: Dict[str, GasField] = {kwargs['name']: GasField(**kwargs) for kwargs in gas_fields_list}

print(gas_fields['Уренгой'].h,gas_fields['Уренгой'].Vg)
print(gas_fields['Ямбургское'].h,gas_fields['Ямбургское'].Vg)
print(gas_fields['Заполярное'].h,gas_fields['Заполярное'].Vg)
print(gas_fields['Медвежье'].h,gas_fields['Медвежье'].Vg)
print(gas_fields['Оренбургское'].h,gas_fields['Оренбургское'].Vg)
print(gas_fields['Вуктыльское'].h,gas_fields['Вуктыльское'].Vg)
print(gas_fields['Шебелинское'].h,gas_fields['Шебелинское'].Vg)
print(gas_fields['Газлинское'].h,gas_fields['Газлинское'].Vg)
print(gas_fields['Астраханское'].h,gas_fields['Астраханское'].Vg)

2544.788451647697 1.01041981979494
2546.2316018256347 1.01102251064758
2551.5016232329126 1.01313192863182
2559.1542079881738 1.0162427676268089
2583.1339147376166 1.026330635707906
2732.4198502712843 1.08761776746044
2564.64916208191 1.018493186367676
2555.8473798148475 1.01493249665487
2577.30357823895 1.0239036549872198
